In [40]:
# from sklearn_pandas import DataFrameMapper, cross_val_score
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
import lda
import sklearn.preprocessing, sklearn.decomposition, sklearn.linear_model, sklearn.pipeline, sklearn.metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import DBSCAN, KMeans
from sklearn import metrics
from sklearn.decomposition import PCA,SparsePCA, TruncatedSVD, NMF
from sklearn.preprocessing import StandardScaler
import matplotlib.cm as cm
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.impute import SimpleImputer
import random

ModuleNotFoundError: No module named 'scipy.sparse.csr_matrix'

In [3]:
# =======================+=======================+=======================+=======================+
movies = pd.read_csv('./netflix-prize-data/movie_titles.csv', encoding = "ISO-8859-1", header = None, names = ['movieId', 'Year', 'Name1', 'Name2', 'Name3', 'Name4'])
for ind, row in movies.iterrows():
    movies.loc[ind, ['title']] = ",".join([str(title) if (str(title) != "nan") else "" for title in movies.loc[ind, ['Name1', 'Name2', 'Name3', 'Name4']]]).rstrip(",")

movies.drop(labels=['Name1', 'Name2', 'Name3', 'Name4'], axis=1, inplace=True)
# movies.set_index('movieId', inplace = True)

Run below cell for preprocessing data or when running for the first time

In [4]:
# # Skip date
# df1 = pd.read_csv('./netflix-prize-data/combined_data_1.txt', header = None, names = ['userId','rating', 'Date'], usecols = [0,1, 2])
# df1['rating'] = df1['rating'].astype(float)

# ratings = df1
# # =======================+=======================+=======================+=======================+
# df_nan = pd.DataFrame(pd.isnull(ratings.rating))
# df_nan = df_nan[df_nan['rating'] == True]
# df_nan = df_nan.reset_index()

# movie_np = []
# movieId = 1
# movie_titles = []


# for i,j in zip(df_nan['index'][1:],df_nan['index'][:-1]):

#     temp = np.full((1,i-j-1), movieId)\
#     movie_np = np.append(movie_np, temp)
#     movieId += 1

# last_record = np.full((1,len(ratings) - df_nan.iloc[-1, 0] - 1),movieId)
# movie_np = np.append(movie_np, last_record)
# ratings = ratings[pd.notnull(ratings['rating'])]

# ratings['movieId'] = movie_np.astype(int)
# ratings['userId'] = ratings['userId'].astype(int)
# print('-Dataset examples-')
# print(ratings.iloc[::5000000, :])
# # =======================+=======================+=======================+=======================+
# ratings.to_csv("./netflix-prize-data/pre-processed/user_movie_rating.csv", index=False)

In [5]:

ratings = pd.read_csv('./netflix-prize-data/pre-processed/user_movie_rating.csv', low_memory=False)

In [6]:
ratings.head()

,userId,rating,Date,movieId
0,1488844,3.0,2005-09-06,1
1,822109,5.0,2005-05-13,1
2,885013,4.0,2005-10-19,1
3,30878,4.0,2005-12-26,1
4,823519,3.0,2004-05-03,1


In [7]:
movies_percentile = 0.7
#Movies rate count percentile
#I will leave only movies on the (1-movies_percentile) percentile with respect to movies rating count

customers_percentile = 0.7
#Customers rate count percentile
#I will leave only customers on the (1-customers_percentile) percentile with respect to customers rating count


df_movie_summary = ratings.groupby('movieId')['rating'].agg(['count'])
df_movie_summary.index = df_movie_summary.index.map(int)
movie_benchmark = round(df_movie_summary['count'].quantile(movies_percentile),0)
drop_movie_list = df_movie_summary[df_movie_summary['count'] < movie_benchmark].index

df_cust_summary = ratings.groupby('userId')['rating'].agg(['count'])
df_cust_summary.index = df_cust_summary.index.map(int)
cust_benchmark = round(df_cust_summary['count'].quantile(customers_percentile),0)
drop_cust_list = df_cust_summary[df_cust_summary['count'] < cust_benchmark].index

print('Movies minimum rating count: {}'.format(movie_benchmark))
print('Customers minimum rating count: {}'.format(cust_benchmark))

print('Original Shape: {}'.format(ratings.shape))
ratings = ratings[~ratings['movieId'].isin(drop_movie_list)]
ratings = ratings[~ratings['userId'].isin(drop_cust_list)]

print('After Trim Shape: {}'.format(ratings.shape))

print('unique movies left:')
print(ratings['movieId'].unique().size)
print('unique customers left:')
print(ratings['userId'].unique().size)

Movies minimum rating count: 1799.0
Customers minimum rating count: 52.0
Original Shape: (24053764, 4)
After Trim Shape: (17337458, 4)
unique movies left:
1350
unique customers left:
143458


In [8]:
# df_p = pd.pivot_table(df,values='rating',index='userId',columns='movieId')
# df_p = df_p.fillna(0)
# print(df_p.head(10))

In [9]:
# Merge the two tables then pivot so we have Users X Movies dataframe
ratings_title = pd.merge(ratings, movies[['movieId', 'title']], on='movieId' )
user_movie_ratings = pd.pivot_table(ratings_title, index='userId', columns= 'title', values='rating')

print('dataset dimensions: ', user_movie_ratings.shape, '\n\nSubset example:')
user_movie_ratings.iloc[:6, :10]

dataset dimensions:  (143458, 1342) 

Subset example:


title,10,10 Things I Hate About You,101 Dalmatians II: Patch's London Adventure,11:14,13 Ghosts,18 Again,1984,2 Fast 2 Furious,200 Cigarettes,2010: The Year We Make Contact
userId,,,,,,,,,,
6,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN
79,NaN,NaN,3.0,NaN,NaN,NaN,NaN,2.0,NaN,NaN
97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
134,NaN,5.0,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN
169,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


To have sklearn run LDA topic modelling to a dataset with missing values like this, we will first cast it to the sparse csr matrix type defined in the SciPi library.

In [10]:
sparse_df = user_movie_ratings.astype(pd.SparseDtype("float64",fill_value=0))
sparse_ratings = csr_matrix(sparse_df.sparse.to_coo())

In [11]:
sparse_ratings.data[np.isnan(sparse_ratings.data)] = 0.0

In [41]:

sparse_df = pd.DataFrame(data=csr_matrix.todense(sparse_ratings))
sparse_df.to_csv('./netflix-prize-data/pre-processed/sparse_ratings.csv', index=False)

# LDA

LDA introduces a latent variable (topics) 'z' that follows a categorical distribution of size k (hyperparmeter). 

It decomposes "User x Movie" (dim: U x M) sparse matrix into "User x Topics" (dim: U x k) and "Topics x Movie" (dim: k x M) matrices.

$$ \sum_{i=1}^{M} P( Movie=i | Topic) = 1, $$

AND

$$ \sum_{i=1}^{U} P(Topic=k | User) = 1, $$

In [12]:
n_topics = 20
mdl_lda = lda.LDA(n_topics=n_topics, n_iter=1000, random_state=1)
mdl_lda.fit(sparse_ratings.toarray().astype(int))

INFO:lda:n_documents: 143458
INFO:lda:vocab_size: 1342
INFO:lda:n_words: 62183611
INFO:lda:n_topics: 20
INFO:lda:n_iter: 1000
INFO:lda:<0> log likelihood: -600162397
INFO:lda:<10> log likelihood: -517238343
INFO:lda:<20> log likelihood: -458753450
INFO:lda:<30> log likelihood: -449773760
INFO:lda:<40> log likelihood: -446622757
INFO:lda:<50> log likelihood: -444989775
INFO:lda:<60> log likelihood: -444042024
INFO:lda:<70> log likelihood: -443266219
INFO:lda:<80> log likelihood: -442666228
INFO:lda:<90> log likelihood: -442192424
INFO:lda:<100> log likelihood: -441848324
INFO:lda:<110> log likelihood: -441558082
INFO:lda:<120> log likelihood: -441330907
INFO:lda:<130> log likelihood: -441136830
INFO:lda:<140> log likelihood: -440955393
INFO:lda:<150> log likelihood: -440775815
INFO:lda:<160> log likelihood: -440667228
INFO:lda:<170> log likelihood: -440540266
INFO:lda:<180> log likelihood: -440418044
INFO:lda:<190> log likelihood: -440287948
INFO:lda:<200> log likelihood: -440206834
INF

In [13]:
topic_word = mdl_lda.topic_word_
n_top_words = 10
topic_words = []

Get top 10 rated movies in each topic

In [14]:
# for i, topic_dist in enumerate(topic_word):
#     topic_words.append(np.array(user_movie_ratings.columns)[np.argsort(topic_dist)][:-(n_top_words+1):-1])
#     print('Topic {}: {}'.format(i, ' '.join(topic_words[i])))

for i, topic_dist in enumerate(topic_word):
    topic_words.append(np.array(user_movie_ratings.columns)[np.argsort(topic_dist)][::-1])
    # print('Topic {}: {}'.format(i, ' '.join(topic_words[i])))

In [15]:
# skl_lda = LatentDirichletAllocation(n_components=3, random_state=1)
# skl_lda.fit(sparse_ratings)

## LDA measures

In [16]:
# `topic_word_` :
#         Alias for `components_`
#     `nzw_` : array, shape = [n_topics, n_features]
#         Matrix of counts recording topic-word assignments in final iteration.
#     `ndz_` : array, shape = [n_samples, n_topics]
#         Matrix of counts recording document-topic assignments in final iteration.
#     `doc_topic_` : array, shape = [n_samples, n_features]
#         Point estimate of the document-topic distributions (Theta in literature)
#     `nz_` : array, shape = [n_topics]
#         Array of topic assignment counts in final iteration.

print(f"topic_word (cluster-movie) distributions:\n{mdl_lda.topic_word_}")
print(f"topic-word (cluster-movie) assignments :\n{mdl_lda.nzw_}")
print(f"document-topic (user-cluster) assignments :\n{mdl_lda.ndz_}")
print(f"document-topic (user-cluster) distributions :\n{mdl_lda.doc_topic_}")
print(f"topic assignment counts :\n{mdl_lda.nz_}")



topic_word (cluster-movie) distributions:
[[1.78314262e-03 4.41917771e-09 4.41917771e-09 ... 4.41917771e-09
  4.41917771e-09 8.09597775e-04]
 [2.74542793e-09 2.74542793e-09 2.74542793e-09 ... 2.74542793e-09
  2.74542793e-09 2.74542793e-09]
 [3.20737293e-09 5.22513445e-03 3.20737293e-09 ... 3.20737293e-09
  1.45938676e-04 3.20737293e-09]
 ...
 [2.65771730e-09 5.60486268e-03 2.65771730e-09 ... 2.65771730e-09
  2.65771730e-09 2.65771730e-09]
 [2.15727044e-09 4.10981808e-03 2.15727044e-09 ... 2.15727044e-09
  2.15727044e-09 2.15727044e-09]
 [3.46396233e-09 2.37420325e-03 6.19807126e-03 ... 3.46396233e-09
  1.15939166e-03 3.46396233e-09]]
topic-word (cluster-movie) assignments :
[[ 4035     0     0 ...     0     0  1832]
 [    0     0     0 ...     0     0     0]
 [    0 16291     0 ...     0   455     0]
 ...
 [    0 21089     0 ...     0     0     0]
 [    0 19051     0 ...     0     0     0]
 [    0  6854 17893 ...     0  3347     0]]
document-topic (user-cluster) assignments :
[[ 46   0

In [17]:
# rat_corr = ratings.corrwith(pd.DataFrame(mdl_lda.doc_topic_), axis=1)
sum(mdl_lda.nz_)

62183611

In [18]:
# mov_cor = movies.corrwith(pd.DataFrame(mdl_lda.topic_word_), axis=0)

In [19]:
# mov_cor.info()
movies.shape
# ratings.shape
# mdl_lda.topic_word_.shape
mdl_lda.doc_topic_.shape

(143458, 20)

In [20]:
user_movie_ratings.tail()

title,10,10 Things I Hate About You,101 Dalmatians II: Patch's London Adventure,11:14,13 Ghosts,18 Again,1984,2 Fast 2 Furious,200 Cigarettes,2010: The Year We Make Contact,...,Xena: Warrior Princess: Season 3,Xena: Warrior Princess: Series Finale,Y Tu Mama Tambien,Yellow Submarine,Yi Yi,Yojimbo,Young Black Stallion,Youngblood,Yu-Gi-Oh!: The Movie,"Zorro, The Gay Blade"
userId,,,,,,,,,,,,,,,,,,,,,
2649370,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN
2649378,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN
2649388,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2649426,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2649429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
#TODO: 
# 1. Find clusters of movies and users based on a probability threshold
# 2. Train classification model to classify users into one of the above 
# clusters based on their similarity measure between different cluster user groups and recommend movies from that group

In [22]:
movie_cluster_dist = pd.DataFrame(topic_word, columns=user_movie_ratings.columns, index=range(n_topics))
user_cluster_dist = pd.DataFrame(mdl_lda.doc_topic_, index=user_movie_ratings.index, columns=range(n_topics))

In [23]:
# sum(movie_cluster_dist.loc[:, 0])
movie_cluster_dist
user_cluster_dist

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
userId,,,,,,,,,,,,,,,,,,,,
6,0.094856,0.000206,0.000206,0.173045,0.057819,0.018724,0.024897,0.000206,0.000206,0.000206,0.000206,0.012551,0.282099,0.000206,0.086626,0.002263,0.000206,0.059877,0.144239,0.041358
7,0.056611,0.051476,0.000128,0.034788,0.000128,0.083569,0.000128,0.000128,0.000128,0.000128,0.000128,0.000128,0.000128,0.362131,0.000128,0.000128,0.131065,0.021951,0.173427,0.083569
79,0.000150,0.000150,0.009173,0.051278,0.067820,0.167068,0.000150,0.000150,0.001654,0.000150,0.000150,0.000150,0.006165,0.143008,0.000150,0.000150,0.152030,0.061805,0.048271,0.290376
97,0.000275,0.058127,0.000275,0.000275,0.008540,0.030579,0.000275,0.000275,0.041598,0.140771,0.000275,0.011295,0.000275,0.157300,0.000275,0.066391,0.317080,0.160055,0.000275,0.005785
134,0.000127,0.000127,0.295063,0.233038,0.102658,0.000127,0.000127,0.007722,0.000127,0.000127,0.000127,0.068481,0.046962,0.000127,0.000127,0.041899,0.120380,0.000127,0.062152,0.020380
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2649370,0.000413,0.132645,0.000413,0.000413,0.000413,0.016942,0.000413,0.000413,0.000413,0.000413,0.000413,0.000413,0.657438,0.025207,0.000413,0.004545,0.000413,0.000413,0.078926,0.078926
2649378,0.027477,0.000450,0.000450,0.000450,0.036486,0.000450,0.000450,0.000450,0.000450,0.022973,0.000450,0.000450,0.347297,0.000450,0.063514,0.018468,0.198649,0.279730,0.000450,0.000450
2649388,0.000389,0.000389,0.000389,0.000389,0.000389,0.004280,0.000389,0.000389,0.000389,0.039300,0.000389,0.074319,0.000389,0.385603,0.000389,0.000389,0.000389,0.490661,0.000389,0.000389


In [24]:
sum(movie_cluster_dist.loc[0])
sum(user_cluster_dist.loc[:, 0])

4288.0577060883215

In [25]:
sum(user_cluster_dist.loc[:, 0])

4288.0577060883215

In [26]:
topic_word_assgn = mdl_lda.nzw_
doc_topic_assgn = mdl_lda.ndz_

In [27]:
movie_cluster_assgn = pd.DataFrame(topic_word_assgn, columns=user_movie_ratings.columns, index=range(n_topics))
user_cluster_assgn = pd.DataFrame(doc_topic_assgn, index=user_movie_ratings.index, columns=range(n_topics))

In [28]:
movie_corr = movie_cluster_assgn.corrwith(movie_cluster_dist, axis=0)
user_corr = user_cluster_assgn.corrwith(user_cluster_dist, axis=1)

In [ ]:
user_cluster_assgn

In [29]:
# movie_cluster_assgn
print(movie_corr)
print(user_corr)

title
10                                             0.981322
10 Things I Hate About You                     0.988866
101 Dalmatians II: Patch's London Adventure    0.999945
11:14                                          0.979529
13 Ghosts                                      0.999360
                                                 ...   
Yojimbo                                        0.999900
Young Black Stallion                           0.992256
Youngblood                                     0.999912
Yu-Gi-Oh!: The Movie                           0.996121
Zorro, The Gay Blade                           0.986450
Length: 1342, dtype: float64
userId
6          1.0
7          1.0
79         1.0
97         1.0
134        1.0
          ... 
2649370    1.0
2649378    1.0
2649388    1.0
2649426    1.0
2649429    1.0
Length: 143458, dtype: float64


In [33]:
# x = 587
# print(user_cluster_assgn.loc[x,:].idxmax())
# print(user_cluster_dist.loc[x,:].idxmax())
# print(len(user_cluster_assgn))
user_cluster_assgn.index

Int64Index([      6,       7,      79,      97,     134,     169,     183,
                188,     195,     199,
            ...
            2649308, 2649328, 2649331, 2649335, 2649336, 2649370, 2649378,
            2649388, 2649426, 2649429],
           dtype='int64', name='userId', length=143458)

In [34]:
lda_predictions = []
for ind, user in enumerate(user_cluster_assgn.index):
    lda_predictions.append(user_cluster_assgn.loc[user,:].idxmax())

In [35]:
lda_predictions = np.array(lda_predictions, dtype=int)
# lda_prediction[:]

In [36]:
np.savetxt("./preds/lda_preds.txt", lda_predictions.astype(int), fmt="%i")

In [38]:
print(movie_cluster_dist.loc[:, "13 Ghosts"])

0     9.280715e-05
1     2.745428e-09
2     1.292574e-03
3     3.002015e-09
4     3.129344e-09
5     2.614195e-09
6     6.322865e-09
7     7.515073e-03
8     9.510373e-09
9     5.008679e-09
10    3.724145e-09
11    3.112515e-09
12    2.110702e-09
13    2.676830e-09
14    3.877031e-09
15    3.961589e-09
16    2.064127e-09
17    2.657717e-09
18    2.157270e-09
19    3.463962e-09
Name: 13 Ghosts, dtype: float64
0       210
1         0
2      4030
3         0
4         0
5         0
6         0
7     19348
8         0
9         0
10        0
11        0
12        0
13        0
14        0
15        0
16        0
17        0
18        0
19        0
Name: 13 Ghosts, dtype: int32


In [39]:
print(movie_cluster_assgn.loc[:, "13 Ghosts"])

0       210
1         0
2      4030
3         0
4         0
5         0
6         0
7     19348
8         0
9         0
10        0
11        0
12        0
13        0
14        0
15        0
16        0
17        0
18        0
19        0
Name: 13 Ghosts, dtype: int32


In [ ]:
# search_str = "Willy Wonka & the Chocolate Factory (1971)"
# for ind, topic in enumerate(topic_words):
#     if search_str in topic:
#         print(f"Found {search_str} in topic {ind}")